```
min  c^t [x y]
a^t [x y] = 0
x in circ
z in simplex
```

In [1]:
import cvxpy as cvx
import numpy as np

In [2]:
N = 5

c = np.random.randn(2*N)
a = np.random.randn(2*N)

x = cvx.Variable(N)
y = cvx.Variable(N)


z = cvx.hstack([x,y])

obj = cvx.Minimize(c@z)
cst = [
    a@z == 0,
    cvx.norm(x) <= 1,
    y >= 0,
    cvx.sum(y) <= 1,
]

prob = cvx.Problem(obj, cst)
prob.solve()

-1.895429631090411

In [3]:
#c.round(2)

In [4]:
x.value.round(2)

array([ 0.63, -0.24,  0.43,  0.44, -0.41])

In [5]:
y.value.round(2)

array([0., 0., 0., 0., 0.])

In [6]:
#z.value.round(2)

```
min  c^t y
z = x + y

other prob:
x in circ
z in simplex
```

In [7]:
import proxie
from proxie import form_stuffers

In [8]:
d = {
    'x': N,
    'y': N,
}

# maybe put all this stuff into an object...
# i like the idea of providing all the tools, so the user can do whatever they want
from_vec, to_vec, slices = form_stuffers(d)

In [9]:
slices

{'x': slice(0, 5, None), 'y': slice(5, 10, None)}

In [10]:
prox1 = proxie.proxes.single_lin_eq.form_prox(c,a)

In [11]:
def prox2(v):
    prox_ball = proxie.proxes.norm2_ball.form_prox(np.zeros(N))
    prox_plex = proxie.proxes.relaxed_simplex.prox
    
    d = from_vec(v)
    
    x = d['x']
    y = d['y']
    
    x = prox_ball(x)
    y = prox_plex(y)
    
    # v = v.copy()
    v[slices['x']] = x
    v[slices['y']] = y
    
    return v

In [12]:
# but we need the pre-defined from_vec and to_vec functions...
from functools import wraps


def vec_wrapper(func):

    @wraps(func)
    def wrapper(x):
        inputs = from_vec(x)
        outputs = func(**inputs)
        x = to_vec(**outputs)

        return x

    
    return wrapper

In [13]:
@vec_wrapper
def prox2(x, y):
    prox_ball = proxie.proxes.norm2_ball.form_prox(np.zeros(N))
    prox_plex = proxie.proxes.relaxed_simplex.prox
    
    x = prox_ball(x)
    y = prox_plex(y)

    return {'x': x, 'y': y}

In [14]:
z, info = proxie.dr.solver(prox1, prox2, np.zeros(2*N), tol=1e-10)

In [15]:
info

SolverInfo(converged=True, exit_tol=9.9e-11, num_iters=89,...)

In [16]:
from_vec(z.round(2))

{'x': array([ 0.63, -0.24,  0.43,  0.44, -0.41]),
 'y': array([0., 0., 0., 0., 0.])}